# Download data

The Xeno-Canto website provides an online form for querying the audio database and downloading the results as a csv file, which includes the id's of the recordings (but not the recordings as such). 

The database was queried for recordings of goshawks as the foreground species, with a quality rating of A. The result set has 186 entries (https://xeno-canto.org/set/8005).

In [1]:
import pandas as pd
from os.path import exists

filename_original = './data/Accipiter_gentilis_xeno_canto_20220921.csv'
if not exists(filename_original):
    # download the query results
    url = 'https://xeno-canto.org/explore/csv?query=accipiter%20gentilis%20q:A'
    df_original = pd.read_csv(url)

    # save the original data set to file
    df_original.to_csv(filename_original)

else:
    df_original = pd.read_csv(filename_original)

NameError: name 'df_original' is not defined

## Augment the data
The downloaded data was augmented with columns for month, stage in the breeding cycle (table 1), sex and life stage (adult/juvenile/fledgling).

In [ ]:
import datetime

# add columns with the month and the corresponding stage in the breeding cycle, as in table 1
df = df_original

def get_breeding_stage(date):
    breeding_stages = {
        9:'Non-breeding', 10:'Non-breeding', 11:'Non-breeding', 12:'Non-breeding', 1:'Non-breeding',
        2:'Territory-building, courtship', 3:'Territory-building, courtship',
        4:'Incubation and nesting', 5:'Incubation and nesting', 6:'Incubation and nesting',
        7:'Fledging', 8:'Fledging'
    }
    month = get_month(date)
    if month:
        return breeding_stages[month]
    else:
        return None

def get_month(date):
    try:
        month = datetime.datetime.strptime(date, "%Y-%m-%d").month
        return month
    except:
        return None

df['month'] = df['Date'].apply(get_month)
df['breeding stage'] = df['Date'].apply(get_breeding_stage)

In [ ]:
# add a columns with life stage and sex
def get_sex(songtype):
    if 'male' in songtype:
        return 'male'
    elif 'female' in songtype:
        return 'female'
    else:
        return None 

def get_life_stage(songtype):
    if 'adult' in songtype:
        return 'adult'
    elif 'juvenile' in songtype:
        return 'juvenile'
    elif 'fledgling' in songtype:
        return 'fledgling'
    
df['sex'] = df['Songtype'].apply(get_sex)
df['life_stage'] = df['Songtype'].apply(get_life_stage)

In [ ]:
# save the augmented data
df.to_csv('data/Accipiter_gentilis.csv')

## Download audio files

In [ ]:
from os.path import exists
import requests

def download_file(url, filename):
    r = requests.get(url)
    with open(filename,'wb') as output_file:
        output_file.write(r.content)

for cat_num in df['Catalogue number']:
    url = 'https://xeno-canto.org/%s/download' % cat_num
    filename = './data/audio/%s.mp3' % cat_num
    if not exists(filename):
        download_file(url, filename)

In [ ]:
df.head(3)